In [ ]:
import gc
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

from scipy.io import loadmat
from shapely.geometry import Point
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import TensorDataset, DataLoader
from torchinfo import summary

%matplotlib inline

Set seeds for reproducability

In [ ]:
np.random.seed(0)
torch.manual_seed(0)
gc.collect()
torch.cuda.empty_cache()

Use CUDA if available

In [ ]:
device = torch.device("cpu")

In [ ]:
foo_dspl_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/train/trainData104/foo_dspl'
foo_dsplRadial_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/train/trainData104/foo_dsplRadial'
foo_trac_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/train/trainData104/foo_trac'
foo_tracRadial_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/train/trainData104/foo_tracRadial'

In [ ]:
def data_to_npArrays(dspl_path, dsplRadial_path, trac_path, tracRadial_path):
    number_samples = len([name for name in os.listdir(dspl_path) if os.path.isfile(os.path.join(dspl_path, name))])
    number_radials = len([name for name in os.listdir(dsplRadial_path) if os.path.isfile(os.path.join(dsplRadial_path, name))])
    
    # save all samples in matrix
    samples = [] 
    for i, filename in enumerate(os.listdir(dspl_path)):
        f = os.path.join(dspl_path, filename)
        if os.path.isfile(f):
            sample = loadmat(f)
            if '__header__' in sample: del sample['__header__']
            if '__version__' in sample: del sample['__version__']
            if '__globals__' in sample: del sample['__globals__']
            sample['name'] = filename
            samples = np.append(samples, sample)
        else:
            continue
    samples = np.array(samples)

    # save all radial patterns of displacements in matrix
    dspl_radials = []
    for i, filename in enumerate(os.listdir(dsplRadial_path)):
        f = os.path.join(dsplRadial_path, filename)
        if os.path.isfile(f):
            radial = loadmat(f)
            if '__header__' in radial: del radial['__header__']
            if '__version__' in radial: del radial['__version__']
            if '__globals__' in radial: del radial['__globals__']
            radial['name'] = filename
            dspl_radials = np.append(dspl_radials, radial)
        else:
            continue
    dspl_radials = np.array(dspl_radials)
    
    # save all targets in matrix
    targets = []
    for i, filename in enumerate(os.listdir(trac_path)):
        f = os.path.join(trac_path, filename)
        if os.path.isfile(f):
            target = loadmat(f)
            if '__header__' in target: del target['__header__']
            if '__version__' in target: del target['__version__']
            if '__globals__' in target: del target['__globals__']
            target['name'] = filename
            targets = np.append(targets, target)
        else:
            continue 
    targets = np.array(targets)
    
    # save all radial patterns of traction forces in matrix
    trac_radials = []
    for i, filename in enumerate(os.listdir(tracRadial_path)):
        f = os.path.join(tracRadial_path, filename)
        if os.path.isfile(f):
            radial = loadmat(f)
            if '__header__' in radial: del radial['__header__']
            if '__version__' in radial: del radial['__version__']
            if '__globals__' in radial: del radial['__globals__']
            radial['name'] = filename
            trac_radials = np.append(trac_radials, radial)
        else:
            continue
    trac_radials = np.array(trac_radials)

    return samples, dspl_radials, targets, trac_radials

In [ ]:
samples, dspl_radials, targets, trac_radials = data_to_npArrays(foo_dspl_path, foo_dsplRadial_path, foo_trac_path, foo_tracRadial_path)
#samples, targets = np.append(samples, dspl_radials), np.append(targets, trac_radials)

Compute loss between sample and target

In [ ]:
# Currently, X and y are of shape (samples, width, height, depth)
X, y = np.array([sample['dspl'] for sample in samples]), np.array([target['trac'] for target in targets])
# Reshape to (samples, channels, depth, height, width)
X = np.moveaxis(X[:, np.newaxis], [2, 3, 4], [-1, 3, 2])
y = np.moveaxis(y[:, np.newaxis], [2, 3, 4], [-1, 3, 2])

Normalize train and validation data

x_min = X.min(axis=(3, 4), keepdims=True)
x_max = X.max(axis=(3, 4), keepdims=True)

X = (X - x_min)/(x_max-x_min)

y_min = y.min(axis=(3, 4), keepdims=True)
y_max = y.max(axis=(3, 4), keepdims=True)

y = (y - y_min)/(y_max-y_min)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
X_train = torch.from_numpy(X).double().to(device)
X_val = torch.from_numpy(X_val).double().to(device)
y_train = torch.from_numpy(y).double().to(device)
y_val = torch.from_numpy(y_val).double().to(device)

In [ ]:
train_set = TensorDataset(X_train, y_train)
val_set = TensorDataset(X_val, y_val)

batch_size = 1

dataloaders = {}
dataloaders['train'] = DataLoader(train_set, batch_size=batch_size, shuffle=True)
dataloaders['val'] = DataLoader(val_set, batch_size=2 * batch_size)

In [ ]:
class ConvBlock_1(nn.Module):
    """Conv3D -> BatchNorm -> ReLU"""
    
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=(2,3,3), padding='same'),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv_block_1(x)

In [ ]:
class ConvBlock_2(nn.Module):
    """Conv3D -> ReLU"""
    
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_block_2 = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=(2,3,3), padding='same'),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv_block_2(x)

In [ ]:
class TracNet(nn.Module):
    def __init__(self, n_channels):
        super().__init__()
        
        self.s1 = ConvBlock_1(n_channels, 32)
        self.s2 = ConvBlock_2(32, 64)
        self.s3 = nn.MaxPool3d(kernel_size=(1, 2, 2))
        self.s4 = ConvBlock_1(64, 64)
        self.s5 = ConvBlock_2(64, 128)
        self.s6 = nn.MaxPool3d(kernel_size=(1, 2, 2))
        self.s7 = ConvBlock_1(128, 128)
        self.s8 = ConvBlock_2(128, 256)
        self.s9 = nn.MaxPool3d(kernel_size=(1, 2, 2))
        self.s10 = ConvBlock_1(256, 128)
        self.s11 = ConvBlock_1(128, 256)
        self.s12 = nn.ConvTranspose3d(256, 256, kernel_size=(1, 3, 3), stride=(1,2,2))
        #fusion3
        self.s13 = ConvBlock_1(512, 64)
        self.s14 = ConvBlock_1(64, 128)
        self.s15 = nn.ConvTranspose3d(128, 128, kernel_size=(1, 3, 3), stride=(1, 2, 2))
        #fusion2
        self.s16 = ConvBlock_1(256, 32)
        self.s17 = ConvBlock_1(32, 64)
        self.s18 = nn.ConvTranspose3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 2, 2))
        #fusion1
        self.s19 = ConvBlock_1(128, 1)
        self.s20 = ConvBlock_1(1, 32)
        self.s21 = nn.Conv3d(32, 1, kernel_size=(2, 3, 3), padding='same')
        
    def forward(self, x):
        x1 = self.s1(x)
        x2 = self.s2(x1)
        x3 = self.s3(x2)
        x4 = self.s4(x3)
        x5 = self.s5(x4)
        x6 = self.s6(x5)
        x7 = self.s7(x6)
        x8 = self.s8(x7)
        x9 = self.s9(x8)
        x10 = self.s10(x9) 
        x11 = self.s11(x10)
        x12 = self.s12(x11)
        padded = torch.nn.functional.pad(x12, (0,-1,0,-1), 'constant', 0)
        fusion3 = torch.cat((x8, padded), dim=1)
        x13 = self.s13(fusion3)
        x14 = self.s14(x13)
        x15 = self.s15(x14)
        padded = torch.nn.functional.pad(x15, (0,-1,0,-1), 'constant', 0)
        fusion2 = torch.cat((x5, padded), dim=1)
        x16 = self.s16(fusion2)
        x17 = self.s17(x16)
        x18 = self.s18(x17)
        padded = torch.nn.functional.pad(x18, (0,-1,0,-1), 'constant', 0)
        fusion1 = torch.cat((x2, padded), dim=1)
        x19 = self.s19(fusion1)
        x20 = self.s20(x19)
        logits = self.s21(x20)
        return logits


In [ ]:
def initialize_weight(module):
    if isinstance(module, (nn.Conv3d, nn.ConvTranspose3d)):
        nn.init.xavier_normal_(module.weight)

In [ ]:
class Custom_MSE(nn.Module):
    def __init__(self):
        super(Custom_MSE, self).__init__();
    
    def forward(self, predictions, target):
        loss_fn = nn.MSELoss(reduction = 'sum')
        loss = 0.5 * loss_fn(predictions, target)
        return loss

In [ ]:
def fit(model, optimizer, dataloaders, num_epochs):
    model = model.double()
    loss_fn = Custom_MSE()
    for epoch in range(num_epochs):
        # Training
        model.train()
        running_loss = 0
        for xb, yb in dataloaders["train"]:
            logits = model(xb)
            logits = logits.double()     
            loss = loss_fn(logits, yb)
            model.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        avg_train_loss = running_loss / len(dataloaders["train"])
        
        # Evaluation
        model.eval()
        loss = 0
        with torch.no_grad():
            for xb, yb in dataloaders["val"]:
                xb = xb.to(device)
                yb = yb.to(device)
                logits = model(xb)
                logits = logits.double()
                loss += loss_fn.forward(logits, yb)
            avg_val_loss = loss / len(dataloaders["val"])
        #scheduler.step()
        print(f"Epoch {epoch}: train_loss = {avg_train_loss}, test_loss = {avg_val_loss}") 

In [ ]:
model = TracNet(n_channels=1)
print(summary(model, verbose=2))

model.apply(initialize_weight)
optimizer = torch.optim.SGD(model.parameters(), lr=0.0000006, momentum=0.9)
#scheduler = StepLR(optimizer, step_size=10, gamma=0.7943)

fit(model, optimizer, dataloaders, num_epochs=20000)

torch.save(model.state_dict(), '/home/alexrichard/LRZ Sync+Share/ML in Physics/toy_model_weights.pth')

In [ ]:
model = TracNet(n_channels=1).double()
model.load_state_dict(torch.load('/home/alexrichard/LRZ Sync+Share/ML in Physics/toy_model_weights.pth'))
model.eval()

In [ ]:
# Test data for working on Martinsried machine
test_sample = np.array(loadmat('/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/train/trainData104/foo_dspl/MLData001-00.mat')['dspl'])
test_target = np.array(loadmat('/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/train/trainData104/foo_trac/MLData001-00.mat')['trac'])

In [ ]:
test_sample = torch.from_numpy(test_sample).double().to(device)
test_target = torch.from_numpy(test_target).double().to(device)

In [ ]:
def predict(features):
    with torch.no_grad():
        return model(features)

In [ ]:
matlab_prediction = np.array(loadmat('/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/prediction.mat')['ans'])

In [ ]:
ground_truth = np.array(loadmat('/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/train/trainData104/trac/MLData001-00.mat')['trac'])

In [ ]:
ground_truth = torch.from_numpy(ground_truth).double()

In [ ]:
def forward(predictions, target):
    loss_fn = nn.MSELoss(reduction = 'sum')
    loss = 0.5 * loss_fn(predictions, target)
    return loss

In [ ]:
forward(ground_truth, trac_field)